# Pre-processing pipeline for spikeglx sessions, zebra finch
- For every run in the session:
 - Load the recordings
 - Extract wav chan with micrhopohone and make a wav chan with the nidq syn signal
 - Get the sync events for the nidq sync channel
 
 - Do bout detection
 
In another notebook, bout detection is curated
- Left to decide where to:
    - Sort spikes
    - Sync the spikes/lfp/nidq
    - make and plot 'bout rasters'

In [1]:
%matplotlib inline
import os
import glob
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal
import traceback
import warnings

from matplotlib import pyplot as plt
from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)


In [2]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util import sglxutil as sglu
from ceciestunepipe.util import rigutil as ru
from ceciestunepipe.util import wavutil as wu
from ceciestunepipe.util import syncutil as su

from ceciestunepipe.util.sound import boutsearch as bs

from ceciestunepipe.util.spikeextractors import preprocess as pre
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import readSGLX as rsgl
from ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor import spikeglxrecordingextractor as sglex

h5py version > 2.10.0. Some extractors might not work properly. It is recommended to downgrade to version 2.10.0: 
>>> pip install h5py==2.10.0


In [3]:
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.toolkit as st
import spikeinterface.sorters as ss
import spikeinterface.comparison as sc
import spikeinterface.widgets as sw
logger.info('all modules loaded')

2021-11-23 15:53:15,522 root         INFO     all modules loaded


## Session parameters and raw files

#### list all the sessions for this bird

In [5]:
bird = 'z_g19o17_21'
all_bird_sess = et.list_sessions(bird)
logger.info('all sessions for bird are {}'.format(all_bird_sess))

2021-11-23 15:53:48,394 root         INFO     all sessions for bird are ['2021-03-26', '2021-03-27', '2021-03-28', '2021-03-29', '2021-03-30', '2021-03-31', '2021-04-01', '2021-04-02', '2021-04-03', '2021-04-04', '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08', '2021-04-09', '2021-04-10', '2021-04-11', '2021-04-12', '2021-04-13', '2021-04-14', '2021-04-15', '2021-04-16', '2021-04-17', '2021-04-18', '2021-04-19', '2021-04-20', '2021-04-21', '2021-04-22', '2021-04-23', '2021-04-24', '2021-04-25', '2021-04-26', '2021-04-27', '2021-04-28', '2021-04-29', '2021-04-30', '2021-05-01', '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07', '2021-05-08', '2021-05-09', '2021-05-10', '2021-05-11', '2021-05-12', '2021-05-13', '2021-05-14']


### set up bird and sessions parameters
this will define:
- locations of files (for the bird)
- signals and channels to look for in the metadata of the files and in the rig.json parameter file: Note that this have to exist in all of the sessions that will be processed
- 'sess' is unimportant here, but it comes handy if there is need to debug usin a single session

In [6]:
reload(et)
# for one example session

sess_par = {'bird': 'z_g19o17_21',
           'sess': '2021-04-21',
           'probes': ['probe_0'], #probes of interest
           'mic_list': ['microphone_0'], #list of mics of interest, by signal name in rig.json
           'sort': 2, #label for this sort instance
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])

ksort_folder = exp_struct['folders']['ksort']
raw_folder = exp_struct['folders']['sglx']

list all the epochs in a session, to check that it is finding what it has to find

In [7]:
sess_epochs = et.list_sgl_epochs(sess_par)
sess_epochs

2021-11-23 15:55:22,060 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21', 'raw': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21', 'sglx': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx', 'kwik': '/experiment/z_g19o17_21/sglx/kwik/2021-04-21', 'processed': '/mnt/sphere/speech_bci/processed_data/z_g19o17_21/2021-04-21/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx', 'tmp': '/experiment/tmp/tmp', 'msort': '/experiment/tmp/z_g19o17_21/sglx/msort/2021-04-21', 'ksort': '/experiment/tmp/z_g19o17_21/sglx/ksort/2021-04-21/2', 'sort': '/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/2'}, 'files': {'par': '/experiment/tmp/z_g19o17_21/sglx/ksort/2021-04-21/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx/settings.isf', 'rig': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx/rig.json', 'kwd': '/experiment/z_g19o17_21/sg

['first_plug_g0']

#### define pre-processing steps for each epoch and for the session

In [8]:
def preprocess_run(sess_par, exp_struct, epoch):
    # get the recordings
    logger.info('PREPROCESSING sess {} | epoch {}'.format(sess_par['sess'], epoch))
    logger.info('getting extractors')
    sgl_exp_struct = et.sgl_struct(sess_par, epoch)
    run_recs_dict, run_meta_files, files_pd, rig_dict = pre.load_sglx_recordings(sgl_exp_struct, epoch)
    
    # get the microphone to wav
    # get the chans
    mic_list = sess_par['mic_list']
    logger.info('Getting microphone channel(s) {}'.format(mic_list))
    mic_stream = pre.extract_nidq_channels(sess_par, run_recs_dict, rig_dict, mic_list, chan_type='adc')
    # get the sampling rate
    nidq_s_f = run_recs_dict['nidq'].get_sampling_frequency()
    
    mic_file_path = os.path.join(sgl_exp_struct['folders']['derived'], 'wav_mic.wav')
    wav_s_f = wu.save_wav(mic_stream, nidq_s_f, mic_file_path)
    
   
    # get the syn to wav
    # get the chans
    sync_list = ['sync']
    logger.info('Getting sync channel(s) {}'.format(sync_list))
    sync_stream = pre.extract_nidq_channels(sess_par, run_recs_dict, rig_dict, sync_list, chan_type='ttl')
    
    sync_file_path = os.path.join(sgl_exp_struct['folders']['derived'], 'wav_sync.wav')
    wav_s_f = wu.save_wav(sync_stream, nidq_s_f, sync_file_path)
    
    logger.info('Getting sync events from the wav sync channel')
    sync_ev_path = os.path.join(sgl_exp_struct['folders']['derived'], 'wav_sync_evt.npy')
    wav_s_f, x_d, ttl_arr = wu.wav_to_syn(sync_file_path)
    logger.info('saving sync events of the wav channel to {}'.format(sync_ev_path))
    np.save(sync_ev_path, ttl_arr)
    
    t_0_path = os.path.join(sgl_exp_struct['folders']['derived'], 'wav_t0.npy')
    logger.info('saving t0 for wav channel to {}'.format(t_0_path))
    np.save(t_0_path, np.arange(sync_stream.size)/wav_s_f)
    #make the sync dict
    syn_dict = {'s_f': wav_s_f,
           't_0_path': t_0_path,
           'evt_arr_path': sync_ev_path}
    
    syn_dict_path = os.path.join(sgl_exp_struct['folders']['derived'],  '{}_sync_dict.pkl'.format('wav'))
    syn_dict['path'] = syn_dict_path
    logger.info('saving sync dict to ' + syn_dict_path)
    with open(syn_dict_path, 'wb') as pf:
        pickle.dump(syn_dict, pf)

    epoch_dict = {'epoch': epoch,
                'files_pd': files_pd,
                 'recordings': run_recs_dict,
                 'meta': run_meta_files,
                 'rig': rig_dict}
    
    return epoch_dict

    
#one_epoch_dict = preprocess_run(sess_par, exp_struct, sess_epochs[0])

2021-11-23 15:56:20,867 root         INFO     PREPROCESSING sess 2021-04-21 | epoch first_plug_g0
2021-11-23 15:56:20,868 root         INFO     getting extractors
2021-11-23 15:56:20,984 root         INFO     Getting microphone channel(s) ['microphone_0']
2021-11-23 15:56:20,985 ceciestunepipe.util.wavutil INFO     sampling rate 25000
2021-11-23 15:56:20,986 ceciestunepipe.util.wavutil INFO     saving (1, 29929184)-shaped array as wav in /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/wav_mic.wav
2021-11-23 15:56:26,545 ceciestunepipe.util.wavutil INFO     saving (1, 29929184)-shaped array as npy in /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/wav_mic.npy
2021-11-23 15:56:28,171 root         INFO     Getting sync channel(s) ['sync']
2021-11-23 15:56:28,173 ceciestunepipe.util.spikeextractors.extractors.spikeglxrecordingextractor.spikeglxrecordingextractor INFO     getting ttl traces, chan range(0, 7)
2021-11-23 15:56:28,863

In [9]:
### sequentially process all runs of the sessions
def preprocess_session(sess_par: dict):
    logger.info('pre-process all runs of sess ' + sess_par['sess'])
    # get exp struct
    sess_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sort=sess_par['sort'])
    # list the epochs
    sess_epochs = et.list_sgl_epochs(sess_par)
    logger.info('found epochs: {}'.format(sess_epochs))
    # preprocess all epochs
    epoch_dict_list = []
    for i_ep, epoch in enumerate(sess_epochs):
        try:
            exp_struct = et.sgl_struct(sess_par, epoch)
            one_epoch_dict = preprocess_run(sess_par, exp_struct, epoch)
            epoch_dict_list.append(one_epoch_dict)
        except Exception as exc:
            warnings.warn('Error in epoch {}'.format(epoch), UserWarning)
            logger.info(traceback.format_exc)
            logger.info(exc)
            logger.info('Session {} epoch {} could not be preprocessed'.format(sess_par['sess'], epoch))
        
    return epoch_dict_list

all_epoch_list = preprocess_session(sess_par)

2021-11-23 15:58:10,010 root         INFO     pre-process all runs of sess 2021-04-21
2021-11-23 15:58:10,011 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21', 'raw': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21', 'sglx': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx', 'kwik': '/experiment/z_g19o17_21/sglx/kwik/2021-04-21', 'processed': '/mnt/sphere/speech_bci/processed_data/z_g19o17_21/2021-04-21/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx', 'tmp': '/experiment/tmp/tmp', 'msort': '/experiment/tmp/z_g19o17_21/sglx/msort/2021-04-21', 'ksort': '/experiment/tmp/z_g19o17_21/sglx/ksort/2021-04-21/2', 'sort': '/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/2'}, 'files': {'par': '/experiment/tmp/z_g19o17_21/sglx/ksort/2021-04-21/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx/settings.isf', 'rig': '/mnt/sphere/speech_

## Process multiple sessions

In [15]:
sess_list = all_bird_sess
# fist implant, right hemisphere
sess_list = ['2021-04-21', '2021-04-23', '2021-04-24', '2021-04-25', '2021-04-26', '2021-04-27', '2021-04-28', 
             '2021-04-29', '2021-04-30', '2021-05-03']

In [16]:
all_sess_dict = {}

for one_sess in sess_list[:]:
    sess_par['sess'] = one_sess
    preprocess_session(sess_par)

2021-11-23 16:16:20,507 root         INFO     pre-process all runs of sess 2021-04-21
2021-11-23 16:16:20,509 ceciestunepipe.file.bcistructure INFO     {'folders': {'bird': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21', 'raw': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21', 'sglx': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx', 'kwik': '/experiment/z_g19o17_21/sglx/kwik/2021-04-21', 'processed': '/mnt/sphere/speech_bci/processed_data/z_g19o17_21/2021-04-21/sglx', 'derived': '/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx', 'tmp': '/experiment/tmp/tmp', 'msort': '/experiment/tmp/z_g19o17_21/sglx/msort/2021-04-21', 'ksort': '/experiment/tmp/z_g19o17_21/sglx/ksort/2021-04-21/2', 'sort': '/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/2'}, 'files': {'par': '/experiment/tmp/z_g19o17_21/sglx/ksort/2021-04-21/2/params.json', 'set': '/mnt/sphere/speech_bci/raw_data/z_g19o17_21/2021-04-21/sglx/settings.isf', 'rig': '/mnt/sphere/speech_

In [17]:
sess_par

{'bird': 'z_g19o17_21',
 'sess': '2021-05-03',
 'probes': ['probe_0'],
 'mic_list': ['microphone_0'],
 'sort': 2}

In [16]:
# Search bouts

## search bouts for those sessions

In [10]:
from ceciestunepipe.util.sound import boutsearch as bs
from ceciestunepipe.util import wavutil as wu

from joblib import Parallel, delayed
import pickle
import sys

In [18]:
reload(bs)

<module 'ceciestunepipe.util.sound.boutsearch' from '/home/finch/repos/ceciestunepipe/ceciestunepipe/util/sound/boutsearch.py'>

In [11]:
def sess_file_id(f_path):
    n = int(os.path.split(f_path)[1].split('-')[-1].split('.wav')[0])
    return n


def get_all_day_bouts(sess_par: dict, hparams:dict, n_jobs: int=12, ephys_software='sglx', 
                     parallel=True) -> pd.DataFrame:
    
    logger.info('Will search for bouts through all session {}, {}'.format(sess_par['bird'], sess_par['sess']))
    exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software=ephys_software)

    # get all the paths to the wav files of the epochs of the day   
    source_folder = exp_struct['folders']['derived']
    wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
    wav_path_list.sort()
    logger.info('Found {} files'.format(len(wav_path_list)))
    print(wav_path_list)
    
    get_file_bouts = lambda path: bs.get_epoch_bouts(path, hparams)
    # Go parallel through all the paths in the day, get a list of all the pandas dataframes for each file
    if parallel:
        sess_pd_list = Parallel(n_jobs=n_jobs, verbose=100, prefer='threads')(delayed(get_file_bouts)(i) for i in wav_path_list)
    else:
        sess_pd_list = [get_file_bouts(i) for i in wav_path_list]
    
    #concatenate the file and return it, eventually write to a pickle
    sess_bout_pd = pd.concat(sess_pd_list)
    return sess_bout_pd

def save_auto_bouts(sess_bout_pd, sess_par, hparams):
    exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='bouts_sglx')
    #sess_bouts_dir = os.path.join(exp_struct['folders']['derived'], 'bouts_ceciestunepipe')
    sess_bouts_dir = exp_struct['folders']['derived']

    sess_bouts_path = os.path.join(sess_bouts_dir, hparams['bout_auto_file'])
    hparams_pickle_path = os.path.join(sess_bouts_dir, 'bout_search_params.pickle')

    os.makedirs(sess_bouts_dir, exist_ok=True)
    logger.info('saving bouts pandas to ' + sess_bouts_path)
    sess_bout_pd.to_pickle(sess_bouts_path)

    logger.info('saving bout detect parameters dict to ' + hparams_pickle_path)
    with open(hparams_pickle_path, 'wb') as fh:
        pickle.dump(hparams, fh)

In [12]:
## need to enter 'sample_rate' from the file!
hparams = {
    # spectrogram
    'num_freq':1024, #1024# how many channels to use in a spectrogram #
    'preemphasis':0.97, 
    'frame_shift_ms':5, # step size for fft
    'frame_length_ms':10, #128 # frame length for fft FRAME SAMPLES < NUM_FREQ!!!
    'min_level_db':-55, # minimum threshold db for computing spe 
    'ref_level_db':110, # reference db for computing spec
    #'sample_rate':None, # sample rate of your data
    
    # spectrograms
    'mel_filter': False, # should a mel filter be used?
    'num_mels':1024, # how many channels to use in the mel-spectrogram
    'fmin': 500, # low frequency cutoff for mel filter
    'fmax': 12000, # high frequency cutoff for mel filter
    
    # spectrogram inversion
    'max_iters':200,
    'griffin_lim_iters':20,
    'power':1.5,

    # Added for the searching
    'read_wav_fun': wu.read_wav_chan, # function for loading the wav_like_stream (has to returns fs, ndarray)
    'file_order_fun': sess_file_id, # function for extracting the file id within the session
    'min_segment': 30, # Minimum length of supra_threshold to consider a 'syllable' (ms)
    'min_silence': 2000, # Minmum distance between groups of syllables to consider separate bouts (ms)
    'min_bout': 5000, # min bout duration (ms)
    'peak_thresh_rms': 0.55, # threshold (rms) for peak acceptance,
    'thresh_rms': 0.25, # threshold for detection of syllables
    'mean_syl_rms_thresh': 0.3, #threshold for acceptance of mean rms across the syllable (relative to rms of the file)
    'max_bout': 120000, #exclude bouts too long
    'l_p_r_thresh': 100, # threshold for n of len_ms/peaks (typycally about 2-3 syllable spans
    
    'waveform_edges': 1000, #get number of ms before and after the edges of the bout for the waveform sample
    
    'bout_auto_file': 'bout_auto.pickle', # extension for saving the auto found files
    'bout_curated_file': 'bout_checked.pickle', #extension for manually curated files (coming soon)
    }

In [14]:
#all_sessions = sess_list[:2]
all_sessions = ['2021-04-21']

for sess in all_sessions:
    sess_par['sess'] = sess
    sess_bout_pd = get_all_day_bouts(sess_par, hparams, parallel=False)
    save_auto_bouts(sess_bout_pd, sess_par, hparams)
    sess_bouts_folder = os.path.join(exp_struct['folders']['derived'], 'bouts')
    #bouts_to_wavs(sess_bout_pd, sess_par, hparams, sess_bouts_folder)

2021-11-23 15:59:15,873 root         INFO     Will search for bouts through all session z_g19o17_21, 2021-04-21
2021-11-23 15:59:16,019 root         INFO     Found 1 files
2021-11-23 15:59:16,020 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/wav_mic.wav


['/mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/wav_mic.wav']
tu vieja file /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/wav_mic.wav


2021-11-23 15:59:16,024 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 1 chunks


  0%|          | 0/1 [00:00<?, ?it/s]

2021-11-23 15:59:35,972 ceciestunepipe.util.sound.boutsearch INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/bout_search_params.pickle
2021-11-23 15:59:35,977 ceciestunepipe.util.sound.boutsearch INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/sglx/first_plug_g0/bout_auto.pickle
2021-11-23 15:59:36,080 root         INFO     saving bouts pandas to /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/bouts_sglx/bout_auto.pickle
2021-11-23 15:59:36,122 root         INFO     saving bout detect parameters dict to /mnt/sphere/speech_bci/derived_data/z_g19o17_21/2021-04-21/bouts_sglx/bout_search_params.pickle


In [22]:
sess_bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 37
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      103 non-null    int64  
 1   end_ms        103 non-null    int64  
 2   start_sample  103 non-null    int64  
 3   end_sample    103 non-null    int64  
 4   p_step        103 non-null    object 
 5   rms_p         103 non-null    float64
 6   peak_p        103 non-null    float64
 7   bout_check    103 non-null    bool   
 8   file          103 non-null    object 
 9   len_ms        103 non-null    int64  
 10  syl_in        103 non-null    object 
 11  n_syl         103 non-null    int64  
 12  peaks_p       103 non-null    object 
 13  n_peaks       103 non-null    int64  
 14  l_p_ratio     103 non-null    float64
 15  waveform      103 non-null    object 
 16  confusing     103 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 13.1+ KB


In [24]:
np.unique(sess_bout_pd['start_ms']).size

103

# debug

## debug search_bout

In [21]:
## look for a single file
sess = sess_list[0]

exp_struct = et.get_exp_struct(sess_par['bird'], sess, ephys_software='sglx')
source_folder = exp_struct['folders']['derived']
wav_path_list = et.get_sgl_files_epochs(source_folder, file_filter='*wav_mic.wav')
wav_path_list.sort()
logger.info('Found {} files'.format(len(wav_path_list)))
print(wav_path_list)

2021-09-22 15:13:39,371 root         INFO     Found 4 files


['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1255_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/1740_g0/wav_mic.wav', '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/2118_g0/wav_mic.wav']


In [22]:
one_file = wav_path_list[0]

In [ ]:
reload(bs)
epoch_bout_pd, epoch_wav = bs.get_bouts_in_long_file(wav_path_list[0], hparams)

2021-09-22 15:13:45,924 ceciestunepipe.util.sound.boutsearch INFO     Getting bouts for long file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


tu vieja file /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-06-14/sglx/0712_g0/wav_mic.wav


2021-09-22 15:13:45,962 ceciestunepipe.util.sound.boutsearch INFO     splitting file into 5 chunks


  0%|          | 0/5 [00:00<?, ?it/s]